# [1907. Count Salary Categories]('https://leetcode.com/problems/count-salary-categories/description/?envType=study-plan-v2&envId=top-sql-50')

Table: Accounts

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| account_id  | int  |
| income      | int  |
+-------------+------+</pre>
account_id is the primary key (column with unique values) for this table.
Each row contains information about the monthly income for one bank account.


Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:

"Low Salary": All the salaries strictly less than 20000.
"Average Salary": All the salaries in the inclusive range (20000, 50000).
"High Salary": All the salaries strictly greater than 50000.

The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Accounts table:
<pre>+------------+--------+
| account_id | income |
+------------+--------+
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |
+------------+--------+</pre>
Output:
<pre>+----------------+----------------+
| category       | accounts_count |
+----------------+----------------+
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |
+----------------+----------------+</pre>
Explanation:
Low Salary: Account 2.
Average Salary: No accounts.
High Salary: Accounts 3, 6, and 8.

In [1]:
import pandas as pd

# Pandas schema

data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#pyspark schema

spark = SparkSession.builder.getOrCreate()

accounts_df = spark.createDataFrame(accounts)
accounts_df.show(truncate=False)

+----------+------+
|account_id|income|
+----------+------+
|3         |108939|
|2         |12747 |
|8         |87709 |
|6         |91796 |
+----------+------+



In [28]:
#in Dataframe

df=accounts_df\
            .withColumn('category', when(col('income')<20000,lit('Low Salary'))\
                                   .when(col('income').between(20000,50000),lit('Average Salary'))\
                                   .otherwise(lit('High Salary')))\
            .groupBy('category').agg(count('*').alias('accounts_count'))


df2 = spark.createDataFrame([
    ('Low Salary',),
    ('Average Salary',),
    ('High Salary',)
], ['category'])

df2.join(df, on='category', how='left').na.fill(0).show()



+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|   High Salary|             3|
|Average Salary|             0|
+--------------+--------------+



In [25]:
a= accounts_df.withColumn('category', when(col('income')<20000,lit('Low Salary')))\
              .where('category="Low Salary"')\
              .groupBy('category').agg(count('*').alias('accounts_count'))


b=accounts_df.withColumn('category',when(col('income').between(20000,50000),lit('Average Salary')))\
            .where('category="Average Salary"')\
            .groupBy('category').agg(count('*').alias('accounts_count'))


c= accounts_df.withColumn('category', when(col('income')>50000,lit('High Salary')))\
              .where('category="High Salary"')\
              .groupBy('category').agg(count('*').alias('accounts_count'))


df=a.union(b).union(c)

df2 = spark.createDataFrame([
    ('Low Salary',),
    ('Average Salary',),
    ('High Salary',)
], ['category'])

df2.join(df, on='category', how='left').na.fill(0).show()


+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|   High Salary|             3|
|Average Salary|             0|
+--------------+--------------+



In [2]:
# in Spark SQL
accounts_df.createOrReplaceTempView('accounts')
spark.sql('''
select "Low Salary" as category, sum(if( income < 20000, 1,0)) as accounts_count from Accounts
union
select "Average Salary" as category, sum(if( income >= 20000 and income <= 50000, 1,0)) as accounts_count  from Accounts
union
select "High Salary" as category, sum(if( income > 50000, 1,0)) as accounts_count  from Accounts

''').show()

+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+



In [ ]:
spark.stop()